In [210]:
class process_multiple_TLEs:


    def load_TLE(self, TLE_file):
        # Input: TLE file location
        # Output: Dictionary of multiple satellites with 2line elements as values
        #self=self.TLE_file
#         print('Reading TLE file ', TLE_file)
        txtFile = open(TLE_file).read() #read file
        loaded_lines = txtFile.split("\n") #split at every new line
        
        # basic TLE format for each satellite consists of three lines.
        # Line1 = GPS satellite code ID or name known as PRN
        # Line 2 = line num ber 1
        # Line 3 = line number 2 
        #Create a dictionary made of only PRN code as key ([-7:-1]). and keep the next two lines as its values
        dic_satellites_twoLine = { loaded_lines[i][-7:-1] : [loaded_lines[i+1], loaded_lines[i+2]] for i in range(0, len(loaded_lines),3 ) }
        return dic_satellites_twoLine

    def print_TLE_object(self, TLE_Object):
        # Input: TLE file Object of Satrec type 
        # Output: standard output of the content of the tle as orbital parameters
        
        from sys import stdout
        from sgp4.conveniences import dump_satrec

        orbital_elements_print = stdout.writelines(dump_satrec(TLE_Object))
        return orbital_elements_print

    
    def create_sat_object_from_TLE(self, Line1, Line2):
        # Input:
        # Line1 = First line of TLE FILE as a string with seperated with white spaces
        # Line2 = Second line of TLE FILE as a string with seperated with white spaces
        # Output: satellite object of type stardec. ready to be used in SGP4 propagation
        
        from sgp4.api import Satrec

        satellite = Satrec.twoline2rv(Line1, Line2)
        return satellite




In [211]:
#from tle_processor import process_multiple_TLEs
from datetime import datetime
from time import gmtime, strftime
from sgp4.api import jday
from sgp4.api import Satrec
from sgp4.api import SGP4_ERRORS
class m_PShared:
#
# This class hold together some variables and functions used across the code.
# These functions can be accessesed by multiple files acrosss the code base
#   
    def get_gps_sats():
        # Input: void
        # Output: GPS satellites dictionary
        # Each element in the dictionary has a key associated with satellite code name
        # For example key PRN 13 will return an arrya of two. each index is one line of two line string for that satellite
        multiFileProcessor = process_multiple_TLEs()
        gps_sats = multiFileProcessor.load_TLE("deps/GPS.TLE")
        return gps_sats
    
    def get_current_time():
        # Input: void
        # Output: returns current time as seperate elements.
        yyyy = strftime("%Y", gmtime())
        mm = strftime("%m", gmtime())
        dd = strftime("%d", gmtime())
        hh = strftime("%H", gmtime())
        MM = strftime("%M", gmtime())
        ss = strftime("%S", gmtime())

        return yyyy, mm, dd, hh, MM ,ss

    def get_jday_from_date(yyyy, mm, dd, hh, MM ,ss):
        # Input: Date and time in the specifed format
        # Output: Julian day and fraction
        jd, fr = jday(yyyy, mm, dd, hh, MM, ss)
        return jd, fr

    def make_starec_obj_from_tle(line1, line2):
        # Input: TLE string line1 and line 2
        # Output: Starec object
        satellite = Satrec.twoline2rv(line1, line2)
        return satellite
    
    def define_error_code(self,error_code):
        return SGP4_ERRORS.get(error_code)
            





In [212]:
# from shared_data import m_PShared
import pandas
class propagate_SGP4:

    def propagate_from_dic(multiple_tle_data_dic, jday, fraction):
        # Input: 
        # multiple_tle_data_dic = dictionary of satellite IDs and their TLE lines
        # jday = Julian day
        # fraction = fraction of julian day
        #
        # Output: 
        # dictionary of the satellite ID and their corresponding position(x,y,z) and velocity(x,y,z)
        dic_satID_pos_vel={}
        
        for satID, tle_line in multiple_tle_data_dic.items():
            
            tle_line_1 = tle_line[0]
            tle_line_2 = tle_line[1]
            
            m_sat = m_PShared.make_starec_obj_from_tle(tle_line_1, tle_line_2)
            error, r_xyz, v_xyz = m_sat.sgp4(jday, fraction)
            
            if (error == 0):
                dic_satID_pos_vel[satID] = [r_xyz, v_xyz]
            else:
                return m_PShared.define_error_code(error)
   
        return dic_satID_pos_vel

  #  def dataframe(dic_satID_pos_vel):
   #         data=dic_satID_pos_vel
    #        dataframe=pd.DataFrame.from_dict(data, orient='index')
     #       return dataframe    

## Plot

In [298]:
import plotly.express as px
#import plotly.graph_objects as go
import plotly.io as io
#from shared_data import m_PShared
#from porpagate_sgp4 import propagate_SGP4
from skyfield.api import Distance, load, wgs84
from skyfield.positionlib import Geocentric
import numpy as np
import pandas as pd
from pathlib import Path

class plot_orbit:
    """
    This class has one attribute: plot_specific_orbit, in which plots the a 
    satellite's orbit given the satellite id'
    """
    def plot_specific_orbit(orbit_id ,graphType = 'Animation'):
        """
        plot_specific_orbit(orbit_id) plots the specified satellite's orbit ffrom
        the current time to 12 hours. 
        input: orbit_id: The ID of the orbit from tle file
        output: interactive plot plotted the orbit for 12 hours. 
        """
       
        # Get satellite data from tle
        multiple_tle_data_dic = m_PShared.get_gps_sats()
        lat = []; lon = [];
        orbit_key=multiple_tle_data_dic.keys()

        # Define time sequence for an hours
        t_array = np.linspace(0,12,31)
        new_plot_data={"orbit_id":[],"timeFrame":[],"latitude":[],"longitude":[] }

        for orbit in orbit_key:
            lat = []; lon = [];
            
            for time in t_array:

                # Get current date and time and convert to jday and fraction
                yyyy, mm, dd, hh, MM ,ss = m_PShared.get_current_time()
                jday, fraction = m_PShared.get_jday_from_date(float(yyyy), float(mm), float(dd), float(hh)+time, float(MM) ,float(ss))

                # Solve for position and velocity using propagate orbit
                dic_satID_pos_vel = propagate_SGP4.propagate_from_dic(multiple_tle_data_dic, jday, fraction)

                # Convert to lla coordinates using skyfield module
                ts = load.timescale()
                t = ts.now()
                d = Distance(m=[dic_satID_pos_vel[orbit_id][0][0], dic_satID_pos_vel[orbit_id][0][1], dic_satID_pos_vel[orbit_id][0][2]])
                p = Geocentric(d.km, t=t)
                g = wgs84.subpoint(p)

                lat.append(g.latitude.degrees)
                lon.append(g.longitude.degrees)
            new_plot_data["orbit_id"].append(orbit_id)
            new_plot_data["timeFrame"].append(t_array)
            new_plot_data["latitude"].append(lat)
            new_plot_data["longitude"].append(lon)
            
            
        df=pd.DataFrame.from_dict(new_plot_data)
        df.drop('timeFrame', inplace=True, axis=1)
        
        lat  = []
        long = []
#         orbit_key = []
        
        for i in df.latitude.iloc[1]:
            lat.append(i)
            
        for i in df.longitude.iloc[1]:
            long.append(i)
        
        
        df2 = pd.DataFrame(lat)
        
        df3 = pd.DataFrame(long)
        
        df3_1 = pd.DataFrame(orbit_key)
        
        
        df4_1 = pd.concat([df2, df3], axis=1)
        df4 = pd.concat([df4_1, df["orbit_id"]], axis=1)
        
        if(graphType == "GroundTracks"):
            # Ground tracks
            fig = px.line_geo(df4, lat=df4.iloc[:, 0],lon=df4.iloc[:, 1], hover_name= df4.iloc[:, 2], title=orbit_id)
        
        elif(graphType == "Animation"):
            # Scater Animation
            fig = px.line_geo(df4, lat=df4.iloc[:, 0],lon=df4.iloc[:, 1], animation_frame=t_array, markers=True, hover_name= df4.iloc[:, 2], title=orbit_id)
            #Put you own path here
            pwd = Path.cwd()
            fig.write_html(str(pwd)+"/"+orbit_id+".html")
        
        elif(graphType == "3d"):            
            #Projection 3D    
            fig = px.scatter_geo(lon = df4.iloc[:, 1], lat = df4.iloc[:, 0], projection="orthographic", hover_name= df4.iloc[:, 2], title=orbit_id) #3D globe with data points
        
       
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
       
        
        
        fig.show()
        

## Main code

In [301]:
# from shared_data import m_PShared
def main():
    
    # Get the satellite IDs
    multiple_tle_data_dic = m_PShared.get_gps_sats()
    orbit_key=multiple_tle_data_dic.keys()
    
    # key is satellite ID to plot (eg. PRN 13)
    for key in orbit_key:
        plot_orbit.plot_specific_orbit(key, "Animation") #graphType = Animation, 3d, GroundTracks
        plot_orbit.plot_specific_orbit(key, "3d") #graphType = Animation, 3d, GroundTracks
        plot_orbit.plot_specific_orbit(key, "GroundTracks") #graphType = Animation, 3d, GroundTracks
        break
   

if __name__ == "__main__":
     main()